In [1]:
import openai
from dotenv import dotenv_values

config = dotenv_values(".env")

from pydantic import BaseModel, Field
from openai import OpenAI
from enum import Enum, StrEnum
from jinja2 import Template

client = OpenAI(api_key=config["OPENAI_API_KEY"])
model_4o = config["OPENAI_MODEL"]
model_4o_mini = config["OPEN_AI_MODEL_4o_MINI"]
model_o1_preview = config["OPEN_AI_MODEL_o1_PREVIEW"]

In [2]:
class Topics(StrEnum):
    personal_financial_guidance="personal_financial_guidance"
    support_assistance="support_assistance"
    debt_management="debt_management"
    budget_management_tracking="budget_management_tracking"
    financial_goals_planning="financial_goals_planning"
    contact_financial_institution="contact_financial_institution"
    other="other"

class GradingHallucination(BaseModel):
    topic: list[Topics] = Field(description="the topic of the last query provided by the user in the messages list")
    last_query: str = Field(description="the last query provided by the user in the messages list")
    is_hallucination_score : float = Field(description="the score associated to the hallucination detection between 0 and 1, 0 being not an hallucination, 1 being an hallucination")
    is_hallucination_confidence : float = Field(description="the confidence score associated to the hallucination detection score between 0 and 1, 0 being not confident, 1 being confident")
    is_hallucination: bool = Field(description="is the last query an hallucination")

class TopicDetection(BaseModel):
    topic: list[Topics] = Field(description="the topic of the last query provided by the user in the messages list")
    last_query: str = Field(description="the last query provided by the user in the messages list")
    topic_detection_confidence : float = Field(description="the confidence score associated to the topic detection score between 0 and 1, 0 being not confident, 1 being confident")


In [3]:
topic_list = """
- Personalized Financial Guidance: Tailored advice to help users make informed financial decisions.
- 24/7 Support: Continuous availability to assist users with their financial queries and concerns.
- Debt Management: Tools to help users manage and reduce their debts effectively.
- Budget Management and Tracking: Features to create, manage, and track personal budgets.
- Financial Goals Planning: Assistance in setting and achieving financial goals.
- Integration with Financial Institutions: Seamless connection with banks and financial services for real-time data."""

template_str = """
### List of authorized topics ###
{{ topics }}
                
### Instruction: find the topic of the current query. ###

### Additional instructions ###
- use the TopicDetection pydantic model to output your result
"""
template = Template(template_str)
generated_prompts = template.render(topics=topic_list)

In [4]:
print(generated_prompts)


### List of authorized topics ###

- Personalized Financial Guidance: Tailored advice to help users make informed financial decisions.
- 24/7 Support: Continuous availability to assist users with their financial queries and concerns.
- Debt Management: Tools to help users manage and reduce their debts effectively.
- Budget Management and Tracking: Features to create, manage, and track personal budgets.
- Financial Goals Planning: Assistance in setting and achieving financial goals.
- Integration with Financial Institutions: Seamless connection with banks and financial services for real-time data.
                
### Instruction: find the topic of the current query. ###

### Additional instructions ###
- use the TopicDetection pydantic model to output your result


In [5]:
query: str = "can you create a plan for me to pay off my credit card over the next 12 months ?"
# query: str = "I'm interested in buying Treasury bills to add to my portfolio, I have $500 to put towards a new investment and am thinking of buying Treasury bills,  is this a good idea? "
# query: str = "I'm a CPA and occassionally provide investment advice that is incidental to other business activites. Do I need to register? "
# query: str = "I'm trying to save $40,000 for a down payment on a new home in 3 years. I can invest $1,000 at the beginning of each month, and expect to earn 10% compounded monthly on my investments. How much will I have saved in 3 years and will it be enough? "
current_messages = [
    {"role": "system", "content": generated_prompts},
    {"role":"user", "content": f"{query}"}]

In [6]:
# current_messages

In [7]:
completion = client.beta.chat.completions.parse(
    model=model_4o_mini,
    messages = current_messages,
    response_format=TopicDetection,
    temperature=0.0
)

In [8]:
completion.choices[0].message.parsed.model_dump()

{'topic': [<Topics.debt_management: 'debt_management'>],
 'last_query': 'can you create a plan for me to pay off my credit card over the next 12 months ?',
 'topic_detection_confidence': 0.95}

In [9]:
query: str = "Tell me a joke about cats"
current_messages = [
    {"role": "system", "content": generated_prompts},
    {"role":"user", "content": f"{query}"}]

completion = client.beta.chat.completions.parse(
    model=model_4o_mini,
    messages = current_messages,
    response_format=TopicDetection,
    temperature=0.0
)

In [10]:
completion.choices[0].message.parsed.model_dump()

{'topic': [<Topics.other: 'other'>],
 'last_query': 'Tell me a joke about cats',
 'topic_detection_confidence': 0.1}

In [14]:
import pandas as pd
df = pd.read_excel("~/Downloads/AI Training Questions.xlsx")
df.head(2)

,#,Question,Expected Answer,AI Answer,Notes,Source
0,1,I qualify for a hardship withdrawal from my 40...,Hardship withdrawals taken from a 401k plan ar...,The penalty for early withdrawals from a 401(k...,NaN,NaN
1,2,I accidentally contributed more to my IRA than...,The excess contribution is subject to a 6% exc...,If you contribute more to your IRA than the al...,NaN,NaN


In [15]:
def process_query(q: str):
    current_messages = [
        {"role": "system", "content": generated_prompts},
        {"role":"user", "content": f"{q}"}]

    completion = client.beta.chat.completions.parse(
        model=model_4o_mini,
        messages = current_messages,
        response_format=TopicDetection,
        temperature=0.0
    )
    return completion
    

In [16]:
res = []
for q in df["Question"]:
    res.append({'query':q, 'answer':process_query(q=q)})
    

In [17]:
len(res)

21

In [18]:
for r in res:
    print(f"query : {r['query']}")
    print(f"completion : {r['answer'].choices[0].message.parsed.model_dump()}")
    print("")

query : I qualify for a hardship withdrawal from my 401K plan, I know that I'll be subject to a penalty but what is the penalty rate?
completion : {'topic': [<Topics.debt_management: 'debt_management'>], 'last_query': "I qualify for a hardship withdrawal from my 401K plan, I know that I'll be subject to a penalty but what is the penalty rate?", 'topic_detection_confidence': 0.85}

query : I accidentally contributed more to my IRA than is permitted, what will happen? 
completion : {'topic': [<Topics.debt_management: 'debt_management'>], 'last_query': 'I accidentally contributed more to my IRA than is permitted, what will happen?', 'topic_detection_confidence': 0.85}

query : What is the limit on the amount of property that may be gifted tax-free between spouses?
completion : {'topic': [<Topics.other: 'other'>], 'last_query': 'What is the limit on the amount of property that may be gifted tax-free between spouses?', 'topic_detection_confidence': 0.7}

query : Our accountant told us that 

In [27]:
for r in res:
    print(f"{r['query']}, {r['answer'].choices[0].message.parsed.model_dump()['topic'][0]}\n")

I qualify for a hardship withdrawal from my 401K plan, I know that I'll be subject to a penalty but what is the penalty rate?, debt_management

I accidentally contributed more to my IRA than is permitted, what will happen? , debt_management

What is the limit on the amount of property that may be gifted tax-free between spouses?, other

Our accountant told us that the gift tax annual exclusion is $15,000, and my husband and I made a gift of $30,000 from our joint checking account to our adult daughter. Do we need to file a gift tax return?, financial_goals_planning

I'm trying to set up a budget what percentage of my budget would you advise my housing costs to be at a maximum? , budget_management_tracking

Can I use money from a 529 plan to pay for room and board? , financial_goals_planning

I'm trying to save $40,000 for a down payment on a new home in 3 years. I can invest $1,000 at the beginning of each month, and expect to earn 10% compounded monthly on my investments. How much wil

In [ ]:
query: str = "can you create a plan for me to pay off my credit card over the next 12 months ?"
query: str = "I'm interested in buying Treasury bills to add to my portfolio, I have $500 to put towards a new investment and am thinking of buying Treasury bills,  is this a good idea? "
query: str = "I'm a CPA and occassionally provide investment advice that is incidental to other business activites. Do I need to register? "
query: str = "I'm trying to save $40,000 for a down payment on a new home in 3 years. I can invest $1,000 at the beginning of each month, and expect to earn 10% compounded monthly on my investments. How much will I have saved in 3 years and will it be enough? "
current_messages = [{"role": "system", "content": 
"""
### List of authorized topics ###
- Personalized Financial Guidance: Tailored advice to help users make informed financial decisions.
- 24/7 Support: Continuous availability to assist users with their financial queries and concerns.
- Debt Management: Tools to help users manage and reduce their debts effectively.
- Budget Management and Tracking: Features to create, manage, and track personal budgets.
- Financial Goals Planning: Assistance in setting and achieving financial goals.
- Integration with Financial Institutions: Seamless connection with banks and financial services for real-time data.
                
### Instruction: detect if the current content is hallucination. ###
An hallucination is defined if the topic of the sentence is not related to the list of authorized topics:


### Additional instructions ###
- use the GradingHallucination pydantic model to output your result
- use `other` topic if the user last query is not related to any of the above.
"""},
{"role":"user", "content": f"{query}"}]

In [107]:
for r in res:
    print(r.model_dump())

{'topic': [<Topics.life_insurance_inquiry: 'life_insurance_inquiry'>], 'last_query': 'I want to buy life insurance', 'sentiment': [<Sentiment.neutral: 'neutral'>], 'sentiment_score': [0.9], 'summary': 'The user is expressing interest in purchasing life insurance.'}
{'topic': [<Topics.life_insurance_inquiry: 'life_insurance_inquiry'>], 'last_query': 'I want to buy life insurance', 'sentiment': [<Sentiment.neutral: 'neutral'>], 'sentiment_score': [0.9], 'summary': 'The user is interested in purchasing life insurance and is seeking information or assistance regarding the process.'}
{'topic': [<Topics.life_insurance_inquiry: 'life_insurance_inquiry'>], 'last_query': 'I want to buy life insurance', 'sentiment': [<Sentiment.neutral: 'neutral'>], 'sentiment_score': [0.9], 'summary': 'The user is interested in purchasing life insurance and is providing necessary personal information for the process.'}


In [39]:
res[0].model_dump()

{'topic': [<Topics.life_insurance_inquiry: 'life_insurance_inquiry'>],
 'query': 'I want to buy life insurance',
 'sentiment': [<Sentiment.neutral: 'neutral'>],
 'sentiment_score': [0.9],
 'summary': 'The user is interested in purchasing life insurance and is providing necessary personal information for the process.'}

In [41]:
completion.usage

CompletionUsage(completion_tokens=54, prompt_tokens=225, total_tokens=279)

In [57]:
def get_cost(comp):

    usage_dict = dict(comp.usage)
    print(usage_dict)
    for k, v in usage_dict.items():
        input = usage_dict["prompt_tokens"] * cost[model]["input"] / 1_000_000
        output = usage_dict["completion_tokens"] * cost[model]["output"] / 1_000_000
    return input + output

get_cost(completion)


{'completion_tokens': 54, 'prompt_tokens': 225, 'total_tokens': 279}


6.615000000000001e-05

In [53]:
dict(completion.usage)

{'completion_tokens': 54, 'prompt_tokens': 225, 'total_tokens': 279}